In [1]:
# Importació de llibreries per al processament de text,
# reducció de dimensionalitat, classificació i validació

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Càrrega del conjunt de dades pre-processat i reduït
# Es tracta d'un subconjunt del corpus Sentiment140 amb textos i etiquetes

df_reduït = pd.read_csv('twitter_reduït_Dataset.csv', encoding='latin-1')

# Separació de les variables predictives (textos) i de la variable objectiu (sentiment)
X = df_reduït['text'].values
y = df_reduït['target'].values

# Definició d’una llista de paraules buides personalitzades
# Aquestes paraules es filtraran durant la vectorització per no aportar informació rellevant

stopwords_personalitzades = [
    'as', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
    'is', 'it', 'for', 'with', 'that', 'this', 'was', 'be',
    'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
    'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not'
]



In [ ]:
# Definició dels paràmetres per a l’exploració mitjançant grid search
# components_k: nombre de components latents a conservar amb SVD (LSA)
# valors_C: valors del paràmetre de regularització per a regressió logística

components_k = [100]  # nombre de components latents per a LSA (Truncated SVD)
valors_C = [1]        # força de regularització per a la regressió logística

# Inicialització de la llista per emmagatzemar els resultats per cada combinació
resultats = []

# Configuració de la validació creuada (5 particions)
# Aquesta tècnica manté la distribució proporcional de les classes en cada fold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Bucle principal sobre totes les combinacions de k (components LSA) i C (regularització)

for k in components_k:
    for C in valors_C:
        puntuacions = []  # Llista per emmagatzemar AUC-ROC de cada fold

        # Validació creuada estratificada
        for train_idx, test_idx in cv.split(X, y):
            # Separació de textos i etiquetes en entrenament i test
            texts_train = X[train_idx]
            texts_test = X[test_idx]
            y_train = y[train_idx]
            y_test = y[test_idx]

            # Vectorització del text amb TF-IDF
            # max_features: nombre màxim de paraules seleccionades per importància
            # stop_words: llista de paraules buides a eliminar
            # S'ajusta només amb el conjunt d'entrenament per evitar fuites

            vectoritzador = TfidfVectorizer(max_features=10000, stop_words=stopwords_personalitzades)
            X_train_tfidf = vectoritzador.fit_transform(texts_train)
            X_test_tfidf = vectoritzador.transform(texts_test)

            # Reducció de dimensionalitat amb Truncated SVD (LSA)
            # Transforma els vectors TF-IDF en una representació temàtica més compacta

            svd = TruncatedSVD(n_components=k, random_state=42)
            X_train_lsa = svd.fit_transform(X_train_tfidf)
            X_test_lsa = svd.transform(X_test_tfidf)

            # Entrenament d'un model de regressió logística
            # penalty='l2': regularització ridge
            # C: invers de la força de regularització (menor valor → més regularització)
            # class_weight='balanced': ajusta els pesos segons la distribució de classes

            model = LogisticRegression(
                penalty='l2',
                C=C,
                solver='lbfgs',
                max_iter=100,
                random_state=42,
            )
            model.fit(X_train_lsa, y_train)

            # Predicció de probabilitats per a la classe positiva
            # i càlcul de la mètrica AUC-ROC per al fold actual

            probs = model.predict_proba(X_test_lsa)[:, 1]
            auc = roc_auc_score(y_test, probs)
            puntuacions.append(auc)
            print("iteracio")

        # Emmagatzematge dels resultats mitjans per a la combinació actual (k, C)

        resultats.append({
            'k_components': k,
            'C': C,
            'auc_roc_mitjana': np.mean(puntuacions)
        })

        # Impressió dels resultats parcials per fer seguiment
        print(resultats)


iteracio
iteracio
iteracio
iteracio
iteracio
[{'k_components': 100, 'C': 1, 'auc_roc_mitjana': np.float64(0.763131953125)}]
